# Feature engineering with featuretools

In [9]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

## Dataset
An example dataset consisting of three tables:

* `clients`: information about clients at a credit union
* `loans`: previous loans taken out by the clients
* `payments`: payments made/missed on the previous loans

The general problem of feature engineering is taking disparate data, often distributed across multiple tables, and combining it into a single table that can be used for training a machine learning model. `featuretools` has the ability to do this for us, creating many new candidate features with minimal effort. These features are combined into a single table that can then be passed on to our model.

In [10]:
# Read in the data
clients = pd.read_csv('data/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('data/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('data/payments.csv', parse_dates = ['payment_date'])

In [11]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [12]:
loans.sample(10)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
288,32885,other,11142,1,11320,2009-05-29,2010-11-06,0.29
69,46180,other,8643,0,10379,2000-11-03,2003-03-31,1.29
346,39384,cash,11728,0,11700,2007-04-20,2009-11-09,5.78
37,49545,home,10519,1,10425,2001-09-27,2004-03-04,1.14
315,44387,home,14900,0,11614,2011-01-04,2013-02-05,4.75
335,39384,credit,2166,1,10274,2012-02-27,2013-08-30,1.56
321,44387,cash,2559,0,10690,2014-02-24,2016-06-11,3.27
400,41472,home,1270,0,11855,2014-04-17,2015-10-28,9.82
210,26326,home,3597,1,11988,2001-04-14,2003-03-06,2.74
379,38537,credit,9355,1,10408,2004-04-22,2006-02-13,1.64


In [13]:
payments.sample(10)

,loan_id,payment_amount,payment_date,missed
738,10407,863,2011-06-23,1
459,11026,2306,2013-12-20,0
12,10990,2284,2006-04-27,1
3433,10154,1367,2002-03-19,0
3324,10891,1998,2013-05-26,1
2815,11287,2069,2005-12-10,0
2969,11361,1507,2007-09-19,1
3110,10166,1990,2001-01-11,1
2325,11975,510,2003-01-17,0
94,11867,416,2006-05-14,0


### Manual Feature Engineering Examples

In [14]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column
clients['log_income'] = np.log(clients['income'])

clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


In [15]:
# Groupby client id and calculate mean, max, min previous loan size
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [16]:
# Merge with the clients dataframe
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.600000,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.300000,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.850000,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.850000,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.950000,13913,1212
5,39505,2011-10-14,153873,610,10,11.943883,7424.050000,14575,904
6,32726,2006-05-01,235705,730,5,12.370336,6633.263158,14802,851
7,35089,2010-03-01,131176,771,3,11.784295,6939.200000,13194,773
8,35214,2003-08-08,95849,696,8,11.470529,7173.555556,14767,667
9,48177,2008-06-09,190632,769,6,12.158100,7424.368421,14740,659


## Featuretools

Featuretools operates on an idea known as Deep Feature Synthesis. The concept of Deep Feature Synthesis is to use basic building blocks known as feature primitives (like the transformations and aggregations done above) that can be stacked on top of each other to form new features. The depth of a "deep feature" is equal to the number of stacked primitives.

The first part of Featuretools to understand is an `entity`. This is simply a table, or in pandas, a DataFrame. We corral multiple entities into a single object called an `EntitySet`. This is just a large data structure composed of many individual entities and the relationships between them.

### EntitySet

In [17]:
es = ft.EntitySet(id = 'clients')

#### Entities

In [18]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [19]:
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [20]:
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [21]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [22]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

In [23]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

In [24]:
es['clients']

Entity: clients
  Variables:
    client_id (dtype: index)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
    join_month (dtype: numeric)
    log_income (dtype: numeric)
  Shape:
    (Rows: 25, Columns: 6)

#### Relationships

In [25]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

In [26]:
# Relationship between previous loans and previous payments# Relati 
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

### Feature Primitives

In [27]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,trend,aggregation,Calculates the slope of the linear trend of variable overtime.
1,sum,aggregation,Counts the number of elements of a numeric or boolean feature.
2,max,aggregation,Finds the maximum non-null value of a numeric feature.
3,skew,aggregation,Computes the skewness of a data set.
4,count,aggregation,Counts the number of non null values.
5,min,aggregation,Finds the minimum non-null value of a numeric feature.
6,time_since_last,aggregation,Time since last related instance.
7,median,aggregation,Finds the median value of any feature with well-ordered values.
8,mode,aggregation,Finds the most common element in a categorical feature.
9,all,aggregation,Test if all values are 'True'.


In [28]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
19,years,transform,Transform a Timedelta feature into the number of years.
20,hour,transform,Transform a Datetime feature into the hour.
21,mod,transform,Creates a transform feature that divides two features.
22,not,transform,"For each value of the base feature, negates the boolean value."
23,months,transform,Transform a Timedelta feature into the number of months.
24,cum_max,transform,Calculates the max of previous values of an instance for each value in a time-dependent entity.
25,day,transform,Transform a Datetime feature into the day.
26,multiply,transform,Creates a transform feature that multplies two features.
27,isin,transform,"For each value of the base feature, checks whether it is in a provided list."
28,weeks,transform,Transform a Timedelta feature into the number of weeks.


In [29]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['years', 'month', 'subtract', 'divide'])

In [30]:
pd.DataFrame(features['MONTH(joined)'].head())

,MONTH(joined)
client_id,
25707,10
26326,5
26695,8
26945,11
29841,8


In [32]:
features.head()

,income,credit_score,join_month,log_income,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_type),LAST(loans.loan_amount),...,income - log_income / credit_score,MEAN(loans.loan_amount) / MEAN(payments.payment_amount),MEAN(loans.loan_amount) / credit_score - income,LAST(loans.rate) / log_income - income,LAST(payments.payment_amount) / credit_score - join_month,credit_score - join_month / credit_score - income,MAX(loans.loan_amount) / join_month - log_income,log_income - income / MAX(payments.payment_amount),log_income / credit_score - join_month,credit_score / income - log_income
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,10,12.261611,7963.950000,3.477000,13913,9.44,home,2203,...,340.434361,6.757398,-0.037779,-0.000035,0.394435,-0.002898,-6151.808355,-78.184075,0.020068,0.002937
26326,227920,633,5,12.336750,7270.062500,2.517500,13464,6.73,credit,5275,...,360.043702,6.231107,-0.031986,-0.000006,1.484076,-0.002763,-1835.144997,-85.744042,0.019645,0.002777
26695,174532,680,8,12.069863,7824.722222,2.466111,14865,6.51,other,13918,...,256.646956,6.480456,-0.045008,-0.000005,3.101190,-0.003865,-3652.456752,-59.522486,0.017961,0.003896
26945,214516,806,11,12.276140,7125.933333,2.855333,14593,5.65,cash,9249,...,266.133652,6.422808,-0.033344,-0.000013,2.008805,-0.003720,-11435.269248,-77.494120,0.015442,0.003758
29841,38354,523,8,10.554614,9813.000000,3.445000,14837,6.76,home,7223,...,73.314427,6.817266,-0.259390,-0.000133,1.553398,-0.013613,-5807.922213,-13.231003,0.020494,0.013640


### Deep Feature Synthesis

The depth of a feature is simply the number of primitives required to make a feature. So, a feature that relies on a single aggregation would be a deep feature with a depth of 1, a feature that stacks two primitives would have a depth of 2 and so on.

In [34]:
# Show a feature with a depth of 1
pd.DataFrame(features['MEAN(loans.loan_amount)'].head(10))

,MEAN(loans.loan_amount)
client_id,
25707,7963.950000
26326,7270.062500
26695,7824.722222
26945,7125.933333
29841,9813.000000
32726,6633.263158
32885,9920.400000
32961,7882.235294
35089,6939.200000


In [35]:
# Show a feature with a depth of 2
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

,LAST(loans.MEAN(payments.payment_amount))
client_id,
25707,293.500000
26326,977.375000
26695,1769.166667
26945,1598.666667
29841,1125.500000
32726,799.500000
32885,1729.000000
32961,282.600000
35089,110.400000


### Automated Deep Feature Synthesis

In [38]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [39]:
features.iloc[:, 4:].head()

,SUM(loans.loan_amount),SUM(loans.rate),STD(loans.loan_amount),STD(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),SKEW(loans.loan_amount),SKEW(loans.rate),MIN(loans.loan_amount),MIN(loans.rate),...,NUM_UNIQUE(loans.WEEKDAY(loan_end)),MODE(loans.MODE(payments.missed)),MODE(loans.DAY(loan_start)),MODE(loans.DAY(loan_end)),MODE(loans.YEAR(loan_start)),MODE(loans.YEAR(loan_end)),MODE(loans.MONTH(loan_start)),MODE(loans.MONTH(loan_end)),MODE(loans.WEEKDAY(loan_start)),MODE(loans.WEEKDAY(loan_end))
client_id,,,,,,,,,,,,,,,,,,,,,
25707,159279,69.54,4044.418728,2.421285,13913,9.44,-0.172074,0.679118,1212,0.33,...,6,0,27,1,2010,2007,1,8,3,0
26326,116321,40.28,4254.149422,1.991819,13464,6.73,0.135246,1.067853,1164,0.50,...,5,0,6,6,2003,2005,4,7,5,2
26695,140845,44.39,4078.228493,1.517660,14865,6.51,0.154467,0.820060,2389,0.22,...,6,0,3,14,2003,2005,9,4,1,1
26945,106889,42.83,4389.555657,1.564795,14593,5.65,0.156534,-0.001998,653,0.13,...,6,0,16,1,2002,2004,12,5,0,1
29841,176634,62.01,4090.630609,2.063092,14837,6.76,-0.212397,0.050600,2778,0.26,...,7,1,1,15,2005,2007,3,2,5,1
